# Train a CNN

Convolutional neural networks (CNNs) are popular tools for creating automated machine learning classifiers on images or image-like samples. By converting audio into a two-dimensional frequency vs. time representation such as a spectrogram, we can generate image-like samples that can be used to train CNNs. 

This tutorial demonstrates the basic use of OpenSoundscape's `preprocessors` and `cnn` modules for training CNNs and making predictions using CNNs.

Under the hood, OpenSoundscape uses Pytorch for machine learning tasks. By using the class `opensoundscape.ml.cnn.CNN`, you can train and predict with PyTorch's powerful CNN architectures in just a few lines of code. 

## Run this tutorial

This tutorial is more than a reference! It's a Jupyter Notebook which you can run and modify on Google Colab or your own computer.

|Link to tutorial|How to run tutorial|
| :- | :- |
| [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kitzeslab/opensoundscape/blob/master/docs/tutorials/train_cnn.ipynb) | The link opens the tutorial in Google Colab. Uncomment the "installation" line in the first cell to install OpenSoundscape. |
| [![Download via DownGit](https://img.shields.io/badge/GitHub-Download-teal?logo=github)](https://minhaskamal.github.io/DownGit/#/home?url=https://github.com/kitzeslab/opensoundscape/blob/master/docs/tutorials/train_cnn.ipynb) | The link downloads the tutorial file to your computer. Follow the [Jupyter installation instructions](https://opensoundscape.org/en/latest/installation/jupyter.html), then open the tutorial file in Jupyter. |

## Setup

### Import needed packages

In [9]:
# the cnn module provides classes for training/predicting with various types of CNNs
from opensoundscape import CNN

#other utilities and packages
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import random 
import subprocess
from glob import glob
import sklearn

#set up plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'

### Set random seeds

Set manual seeds for Pytorch and Python. These essentially "fix" the results of any stochastic steps in model training, ensuring that training results are reproducible. You probably don't want to do this when you actually train your model, but it's useful for debugging.

In [10]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

### Download files

Training a machine learning model requires some pre-labeled data. These data, in the form of audio recordings or spectrograms, are labeled with whether or not they contain the sound of the species of interest. 

These data can be obtained from online databases such as Xeno-Canto.org, or by labeling one's own ARU data using a program like Cornell's Raven sound analysis software. In this example we are using a set of annotated avian soundscape recordings that were annotated using the software Raven Pro 1.6.4 (Bioacoustics Research Program 2022):

<blockquote><i>An annotated set of audio recordings of Eastern North American birds containing frequency, time, and species information. </i> Lauren M. Chronister,  Tessa A. Rhinehart,  Aidan Place,  Justin Kitzes.
https://doi.org/10.1002/ecy.3329 
</blockquote>

These are the same data that are used by the annotation and preprocessing tutorials, so you can skip this step if you've already downloaded them there.

In [11]:
# Set the current directory to where the dataset is downloaded
dataset_path = Path("Annotations/")

# Make a list of all of the csvs utilized
selection_files = glob(f"{dataset_path}/*/*.csv")
index_to_remove = 1 #this is specifically removing the combined_output_macaulay_v1.csv that i used for experiment 1
selection_files = selection_files[:index_to_remove] + selection_files[index_to_remove+1:]


In [12]:
# if False:
#    masterlist = pd.DataFrame()
#    for i in range(len(selection_files)):
#       templist = pd.read_csv(selection_files[i])
#       templist['annotation_file'] = selection_files[i]
#       masterlist = pd.concat([masterlist, templist])
#    masterlist = masterlist.iloc(0:5)

In [13]:
valid = pd.read_csv(selection_files[1])
train =pd.concat((pd.read_csv(selection_files[0]), pd.read_csv(selection_files[2])))
train['annotation'] = train['annotation'].replace([' KAAM_song','KAAM_wsong'],'KAAM_song')

In [14]:
valid["annotation"].value_counts()
#ensure that all KAAM_song are good and uniform

annotation
KAAM_song     107
0              93
ANIA_song      89
u              72
KAAM_song?     67
ANIA_song?     41
KAAM_ANIA      10
AKEK_song       9
AKEK_song?      8
AKEK_KAAM       8
playback        2
Name: count, dtype: int64

In [15]:
train["annotation"].value_counts()
#ensure that all KAAM_song are good and uniform

annotation
KAAM_song        313
KAAM_call         80
?                 20
?_KAAM_song       16
?_KAAM_call        7
?KAAM_song         6
?KAAM_call         1
  ?_KAAM_song      1
Name: count, dtype: int64

sam = valid

## Prepare audio data

To prepare audio data for machine learning, we need to convert our annotated data into clip-level labels.

These steps are covered in depth in other tutorials, so we'll just set our clip labels up quickly for this example.

First, get exactly matched lists of audio files and their corresponding selection files:

Next, convert the selection files and audio files to a `BoxedAnnotations` object, which contains the time, frequency, and label information for all annotations for every recording in the dataset.

In [16]:
valid

,audio_file,start_time,end_time,AKEK_song,folder,point,notes,annotation
0,/media/kiwi/datasets/finalized/kaua2024a/AKEK_...,93,96,9.070529,A1-0_11883,A1-0,lowers in pitch after two notes,KAAM_song
1,/media/kiwi/datasets/finalized/kaua2024a/AKEK_...,147,150,8.924845,A1-0_11883,A1-0,NaN,KAAM_song
2,/media/kiwi/datasets/finalized/kaua2024a/AKEK_...,78,81,9.457360,A1-0_11883,A1-0,NaN,KAAM_song
3,/media/kiwi/datasets/finalized/kaua2024a/AKEK_...,102,105,10.097325,A1-0_11883,A1-0,NaN,KAAM_song
4,/media/kiwi/datasets/finalized/kaua2024a/AKEK_...,108,111,9.185908,A1-0_11883,A1-0,NaN,KAAM_song
...,...,...,...,...,...,...,...,...
501,/media/kiwi/datasets/finalized/kaua2024a/PUAI_...,150,153,5.530422,OS-UUK-20_11919,OS-UUK-20,NaN,u
502,/media/kiwi/datasets/finalized/kaua2024a/PUAI_...,165,168,3.910681,OS-UUK-20_11919,OS-UUK-20,NaN,u
503,/media/kiwi/datasets/finalized/kaua2024a/PUAI_...,171,174,4.832873,OS-UUK-20_11919,OS-UUK-20,NaN,u
504,/media/kiwi/datasets/finalized/kaua2024a/PUAI_...,9,12,4.613531,OS-UUK-20_11919,OS-UUK-20,NaN,u


In [17]:
from opensoundscape import BoxedAnnotations
valid_boxed = BoxedAnnotations(df = valid, audio_files=valid["audio_file"])
# Create dataframe of one-hot labels
class_sub = ["KAAM_song"]

valid_clips = valid_boxed.clip_labels(
    clip_duration = 3,
    clip_overlap = 0,
    min_label_overlap = 0,
    audio_files = valid["audio_file"].unique(),
    class_subset = class_sub
)
ann = valid_boxed.df[["audio_file", "start_time", "end_time"]]

def overlaps_any(row):
    a = ann[ann.audio_file == row.name[0]]
    return ((a.start_time < row.name[1] + 3) &
            (a.end_time > row.name[1])).any()

valid_clips = valid_clips[valid_clips.apply(overlaps_any, axis=1)]


train_boxed = BoxedAnnotations(df = train, audio_files=train["audio_file"])
train_clips = train_boxed.clip_labels(
    clip_duration = 3, 
    clip_overlap = 1.5,
    min_label_overlap = 0.5,
    audio_files = train["audio_file"].unique(),
    class_subset = class_sub
)

In [18]:
valid_clips["KAAM_song"].value_counts()

KAAM_song
False    398
True     106
Name: count, dtype: int64

## Create train, validation, and test datasets

To train and test a model, we use three datasets:

* The **training dataset** is used to fit your machine learning model to the audio data. 
* The **validation dataset** is a held-out dataset that is used to select hyperparameters (e.g. how many epochs to train for) during training
* The **test dataset** is another held-out dataset that we use to check how the model performs on data that were not available at all during training.

While both the training and validation datasets are used while training the model, the test dataset is never touched until the model is fully trained and completed.

The training and validation datasets may be gathered from the same source as each other. In contrast, the test dataset is often gathered from a different source to assess whether the model's performance generalizes to a real-world problem. For example, training and validation data might be drawn from an online database like Xeno-Canto, whereas the testing data is from your own field data. 

### Create a test dataset

We'll separate the test dataset first. For a good assessment of the model's generalization, we want the test set to be independent of the training and validation datasets. For example, we don't want to use clips from the same source recording in the training dataset and the test dataset.

For this example, we'll use the recordings in the folders `Recording_1`, `Recording_2` and `Recording_3` as our training and validation data, and use the recordings in folder `Recording_4` as our test data. 

In [19]:
if False:
    from sklearn.model_selection import train_test_split

    train_df, val_df = train_test_split(clip_labels,test_size=0.2)
    train_df.shape,val_df.shape


If you wanted, you could load the training and testing set from these saved CSV files.

In [20]:
if False:
    train_and_val_set = pd.read_csv("./annotated_data/train_and_val_set.csv", index_col=[0, 1, 2]
    )
    test_set = pd.read_csv("./annotated_data/test_set.csv", index_col=[0, 1, 2])

### Split training and validation datasets

Now, separate the remaining non-test data into training and validation datasets.

The idea of keeping a separate validation dataset is that, throughout training, we can 'peek' at the performance on the validation set to choose hyperparameters. (This is in contrast to the test dataset, which we will not look at until we've finished training our model.)

One important hyperparameter is the number of **epochs** to train to, in order to prevent overfitting. Each epoch includes one round of fitting on each training sample. 

If a model's performance on a training dataset continues to improve as it trains, but its performance on the validation dataset plateaus, this could incate the model is **overfitting** on the training dataset, learning information specific to those particular samples instead of gaining the ability to generalize to new data.

In [21]:
# # Split our training data into training and validation sets
# train_df, valid_df = sklearn.model_selection.train_test_split(
#     train_and_val_set, test_size=0.1, random_state=0
# )

In [22]:
if False:
    train_clips.to_csv("Experiment4/train_set.csv")
    valid_clips.to_csv("Experiment4/valid_set.csv")

***look back to reorganize file save locations, EDIT THIS EVERY EXPERIMENT***

In [23]:
train_df = pd.read_csv("Experiment4/train_set.csv", index_col=[0,1,2])
val_df = pd.read_csv("Experiment4/valid_set.csv", index_col=[0,1,2])

### Resample data for even class representation

Before training, we will balance the number of samples of each class in the training set. This helps the model learn all of the classes, rather than paying too much attention to the classes with the most labeled annotations. 

In [24]:
train_df.KAAM_song.value_counts()
val_df.KAAM_song.value_counts()

KAAM_song
False    398
True     106
Name: count, dtype: int64

In [25]:
from opensoundscape.data_selection import resample


## Set up model

Now we create a model object. We have to select several parameters when creating this object: its `architecture`, `classes`, and `sample_duration`. 

Some additional parameters can also be changed at this step, such as the preprocessor used to create spectrograms and the shape of the spectrograms. 

For more detail on this step, see the ["Customize CNN training"]("tutorials/CNN.html") tutorial.


### Create CNN object

Now, create a CNN object with this architecture, the classes we put into the dataframe above, and the same sample duration as we selected above.

The first time you run this script for a particular architecture, OpenSoundscape will download the desired architecture.

In [26]:
# Create a CNN object designed to recognize 3-second samples
from opensoundscape import CNN

# Use resnet34 architecture
architecture = "resnet18"

# Can use this code to get your classes, if needed
class_list = list(train_df.columns)
clip_duration = 3

model = CNN(
    architecture=architecture,
    classes=class_list,
    sample_duration=clip_duration,  # 3s, selected above
)

### Check model device

If a GPU is available on your computer, the CNN object automatically selects it for accellerating performance. You can override `.device` to use a specific device such as `cpu` or `cuda:3`

In [27]:
print(f"model.device is: {model.device}")
model.device = "cuda:0"
print(f"model.device is: {model.device}")

model.device is: cuda:0
model.device is: cuda:0


### Set up WandB model logging

While this step is optional, it is very helpful for model training. In this step, we set up model logging on a service called **Weights & Biases** (AKA WandB). 

Weights & Biases is a free website you can use to monitor model training. It is integrated with OpenSoundscape to include helpful functions such as checking on your model's training progress in real time, visualizing the spectrograms created for training your model, comparing multiple tries at training the same model, and more. For more information, check out this [blog post](https://wandb.ai/wandb_fc/repo-spotlight/reports/Community-Spotlight-OpenSoundscape--Vmlldzo0MDcwMTI4). 

The instructions below will help you set up `wandb` logging:

* Create an account on the [Weights and Biases website](https://wandb.ai/). 
* The first time you use `wandb`, you'll need to run `wandb.login()` in Python or `wandb login` on the command line, then enter the API key from your [settings](https://wandb.ai/settings) page
* In a Python script where you want to log model training, use `wandb.init()` as demonstrated below. The "Entity" or team option allows runs and projects to be shared across members in a group, making it easy to collaborate and see progress of other team members' runs.


As training progresses, performance metrics will be plotted to the wandb logging platform and visible on this run's web page. For example, this [wandb web page](https://wandb.ai/kitzeslab/opensoundscape%20training%20demo/runs/w1xyk7zr/workspace?workspace=user-samlapp) shows the content logged to wandb when this notebook was run by the Kitzes Lab. By default, OpenSoundscape + WandB integration creates several pages with information about the model:

- Overview: hyperparameters, run description, and hardware available during the run
- Charts: "Samples" panel with audio and images of preprocessed samples (useful for checking that your preprocessing performs as expected and your labels are correct)
- Charts: graphs of each class's performance metrics over training time
- Model: summary of model architecture
- Logs: standard output of training script
- System: computational performance metrics including memory, CPU use, etc

When training several models and comparing performance, the "Project" page of WandB provides comparisons of metrics and hyperparameters across training runs.

In [28]:
import wandb

try:
    wandb.login()
    wandb_session = wandb.init(
        entity="kitzeslab",  # replace with your entity/group name
        project="KAAM",
        name="experiment4",
    )
except:  # if wandb.init fails, don't use wandb logging
    print("failed to create wandb session. wandb session will be None")
    wandb_session = None

wandb: Currently logged in as: hindsdavis2 (kitzeslab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/dah238/miniconda3/envs/davis/lib/python3.10/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/home/dah238/miniconda3/envs/davis/lib/python3.10/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


## Train the CNN

Finally, train the CNN for two epoch. Typically, we would train the model for more than two epochs, but because training is slow and is much better done outside of a Jupyter Notebook, we just include this as a short demonstration of training.

Each **epoch** is one pass-through of all of the samples in the training dataset, plus running predictions on the validation dataset. 

Each epoch is composed of smaller groups of samples called **batches**. The machine learning model predicts on every sample in the batch, then the model weights are updated based on those samples. Larger batches can increase training speed, but require more memory. If you get a memory error, try reducing the batch size.

We use default training parameters, but many aspects of CNN training can be customized (see the "Customize CNN training" tutorial for examples).

In [29]:
checkpoint_folder = Path("Experiment4/checkpoints")
checkpoint_folder.mkdir(exist_ok=True)
#CHANGE CHECKPOINT FOLDER PATH

In [30]:
%%capture --no-stdout --no-display
# Uncomment the line above to silence outputs from this cell
num_workers = 2
model.train(
    train_df = train_df,
    validation_df = val_df,
    epochs=50, #change to more later
    batch_size=64,
    log_interval=100,  # log progress every 100 batches
    num_workers=num_workers,  # parallelized cpu tasks for preprocessing
    wandb_session=wandb_session,
    save_interval=5,  # save checkpoint every 10 epochs
    save_path=checkpoint_folder,  # location to save checkpoints
)


Training Epoch 0


  0%|          | 0/34 [00:00<?, ?it/s]

[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2400) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2208) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2304) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2336) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2400) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774]

Epoch: 0 [batch 0/34, 0.00%] 
	Epoch Running Average Loss: 0.716
	Most Recent Batch Loss: 0.716


[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2208) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2400) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2208) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2304) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2336) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774]

...

Epoch: 49 [batch 0/34, 0.00%] 
	Epoch Running Average Loss: 0.097
	Most Recent Batch Loss: 0.097


[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2208) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2400) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2208) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2304) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2336) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (2272) too large for available bit count (2200)
[src/libmpg123/layer3.c:INT123_do_layer3():1774]


Validation.


  0%|          | 0/8 [00:00<?, ?it/s]


Best Model Appears at Epoch 14 with Validation score 0.698.


Once this is finished running, you have trained the CNN. 

To generate predictions on audio files using the CNN, use the `.predict()` method of the CNN object. Here, we apply a sigmoid activation layer which maps the CNN's outputs (all real numbers) to a 0-1 range. 

In [31]:
scores_df = model.predict(valid_df.head(), activation_layer="sigmoid")

NameError: name 'valid_df' is not defined

We don't expect this CNN to actually be good at classifying sounds, since we only trained it with a few examples and for a couple epochs. We'd want to train with hundreds of examples per class for 10-100 epochs as a starting point for training a useful model. 

For guidance on how to use machine learning classifiers, see the Classifieres 101 Guide on opensoundscape.org and the tutorial on predicting with pre-trained CNNs.


**Clean up:** Run the following cell to delete the files created in this tutorial. However, these files are used in other tutorials, so you may wish not to delete them just yet.

In [ ]:
import shutil

# uncomment to remove the training files
# shutil.rmtree('./annotated_data')

shutil.rmtree("./wandb")
shutil.rmtree("./model_training_checkpoints")
try:
    Path("annotation_Files.zip").unlink()
except:
    pass
try:
    Path("mp3_Files.zip").unlink()
except:
    pass